In [31]:
import pandas as pd
from sklearn import preprocessing
import sys

import all_22277_list

In [32]:
all_df = pd.read_csv("./"+"H1N1Data"+".txt",sep='\t',encoding='utf-8')

cols=all_22277_list.cols

#1個list存cols 這邊存的是不同資料時間的tag
print(type(cols))


#把DATA利用cols當作列來排好
all_df=all_df[cols]


<class 'list'>


In [33]:
def PreprocessData(raw_df):

    df=raw_df.drop(['ID'], axis=1)
    ndarray = df.values
    Features = ndarray[:,1:]
    Label = ndarray[:,0]

    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    scaledFeatures=minmax_scale.fit_transform(Features)

    return scaledFeatures,Label

In [34]:
#all features是每個人所附帶的資料組
#all label 每筆資料組(人) 的結果1 或 0用來訓練
all_Features,all_Label=PreprocessData(all_df)

'''
前兩筆資料組用來做為測試用
後面全部用來訓練用
'''
test_Features=all_Features[:2]
test_Label=all_Label[:2]
train_Features=all_Features[2:]
train_Label=all_Label[2:]


In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

model = Sequential()
model.add(Dense(units=100, input_dim=22277, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=100, kernel_initializer='uniform', activation='relu'))
model.add(Dense(units=100, kernel_initializer='uniform', activation='relu'))
model.add(Dense(units=100, kernel_initializer='uniform', activation='relu'))
model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
model.summary()
model.save("DeepFlu4Layer.h5")

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 100)               2227800   
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_27 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_28 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 101       
Total params: 2,258,201
Trainable params: 2,258,201
Non-trainable params: 0
____________________________________________

In [36]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
train_history =model.fit(x=train_Features, y=train_Label, validation_split=0.1, epochs=150, batch_size=200,verbose=2)

Epoch 1/150
1/1 - 1s - loss: 0.6932 - accuracy: 0.5278 - val_loss: 0.6962 - val_accuracy: 0.0000e+00
Epoch 2/150
1/1 - 0s - loss: 0.6928 - accuracy: 0.5000 - val_loss: 0.7070 - val_accuracy: 0.0000e+00
Epoch 3/150
1/1 - 0s - loss: 0.6917 - accuracy: 0.5000 - val_loss: 0.7042 - val_accuracy: 0.0000e+00
Epoch 4/150
1/1 - 0s - loss: 0.6895 - accuracy: 0.5833 - val_loss: 0.6910 - val_accuracy: 0.7500
Epoch 5/150
1/1 - 0s - loss: 0.6874 - accuracy: 0.7222 - val_loss: 0.6851 - val_accuracy: 1.0000
Epoch 6/150
1/1 - 0s - loss: 0.6787 - accuracy: 0.8056 - val_loss: 0.6999 - val_accuracy: 0.2500
Epoch 7/150
1/1 - 0s - loss: 0.6694 - accuracy: 0.8333 - val_loss: 0.7043 - val_accuracy: 0.2500
Epoch 8/150
1/1 - 0s - loss: 0.6494 - accuracy: 0.8333 - val_loss: 0.6602 - val_accuracy: 1.0000
Epoch 9/150
1/1 - 0s - loss: 0.6309 - accuracy: 0.9444 - val_loss: 0.7378 - val_accuracy: 0.0000e+00
Epoch 10/150
1/1 - 0s - loss: 0.6114 - accuracy: 0.7778 - val_loss: 0.6490 - val_accuracy: 1.0000
Epoch 11/150


In [37]:
scores=model.evaluate(x=test_Features, y=test_Label)

1/1 [==============================] - 0s 16ms/step - loss: 2.5536 - accuracy: 0.0000e+00


In [38]:
all_Features,Label=PreprocessData(all_df)

#利用網路去預測測試資料
#predict_classes: 取出屬於第幾類
all_probability=model.predict_classes(all_Features)
#predict_proba: 取出每個預測的機率
all_probability_score=model.predict_proba(all_Features)
pd=all_df

#把預測值存到表格
pd.insert(len(all_df.columns), 'probability',all_probability)

'''
前兩個是test DATA
'''
predict=all_probability[:2]
predict_score=all_probability_score[:2]
predict=predict.tolist()
predict_score=predict_score.tolist()

C:\Users\choug\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
C:\Users\choug\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [39]:
f1 = open("./"+"result"+".txt", 'a', encoding = 'UTF-8')
f1.write(str(int(test_Label[0]))+"\t"+str(predict[0])+"\t"+str(predict[1])+"\t"+str(predict_score[0])+"\t"+str(predict_score[1])+"\n")
f1.close


<function TextIOWrapper.close()>